In [1]:
import sqlite3

DB_PATH = "../db/footbase_big5.db"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Base table for match results (Bronze layer)
cursor.execute("""
CREATE TABLE IF NOT EXISTS matches (
    date TEXT,
    home_team TEXT,
    away_team TEXT,
    home_goals INTEGER,
    away_goals INTEGER,
    result TEXT,
    league TEXT,
    season TEXT
);
""")

conn.commit()
conn.close()
print("✅ Clean 'footbase_big5.db' created!")

✅ Clean 'footbase_big5.db' created!


In [3]:
import sqlite3

DB_PATH = "../db/footbase_big5.db"
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Drop existing table (if you’re reloading from scratch)
cursor.execute("DROP TABLE IF EXISTS matches;")

# Create with full column list
cursor.execute("""
CREATE TABLE matches (
    date TEXT,
    home_team TEXT,
    away_team TEXT,
    home_goals INTEGER,
    away_goals INTEGER,
    result TEXT,
    home_shots INTEGER,
    away_shots INTEGER,
    home_shots_on_target INTEGER,
    away_shots_on_target INTEGER,
    odds_home REAL,
    odds_draw REAL,
    odds_away REAL,
    league TEXT,
    season TEXT
);
""")

conn.commit()
conn.close()
print("✅ Table recreated with full schema!")

✅ Table recreated with full schema!


In [4]:
import pandas as pd
import sqlite3
import glob
import os

CSV_DIR = "../data/raw"
DB_PATH = "../db/footbase_big5.db"

conn = sqlite3.connect(DB_PATH)

for league in ["Premier_League", "La_Liga", "Bundesliga", "Serie_A", "Ligue_1"]:
    files = glob.glob(os.path.join(CSV_DIR, league, "*.csv"))
    for file in files:
        df = pd.read_csv(file)
        df["league"] = league.replace("_", " ")
        df.to_sql("matches", conn, if_exists="append", index=False)
        print(f"✅ Loaded {file}")

conn.close()

✅ Loaded ../data/raw\Premier_League\Premier_League_1718.csv
✅ Loaded ../data/raw\Premier_League\Premier_League_1819.csv
✅ Loaded ../data/raw\Premier_League\Premier_League_1920.csv
✅ Loaded ../data/raw\Premier_League\Premier_League_2021.csv
✅ Loaded ../data/raw\Premier_League\Premier_League_2122.csv
✅ Loaded ../data/raw\Premier_League\Premier_League_2223.csv
✅ Loaded ../data/raw\Premier_League\Premier_League_2324.csv
✅ Loaded ../data/raw\Premier_League\Premier_League_2425.csv
✅ Loaded ../data/raw\La_Liga\La_Liga_1718.csv
✅ Loaded ../data/raw\La_Liga\La_Liga_1819.csv
✅ Loaded ../data/raw\La_Liga\La_Liga_1920.csv
✅ Loaded ../data/raw\La_Liga\La_Liga_2021.csv
✅ Loaded ../data/raw\La_Liga\La_Liga_2122.csv
✅ Loaded ../data/raw\La_Liga\La_Liga_2223.csv
✅ Loaded ../data/raw\La_Liga\La_Liga_2324.csv
✅ Loaded ../data/raw\La_Liga\La_Liga_2425.csv
✅ Loaded ../data/raw\Bundesliga\Bundesliga_1718.csv
✅ Loaded ../data/raw\Bundesliga\Bundesliga_1819.csv
✅ Loaded ../data/raw\Bundesliga\Bundesliga_1920.

In [8]:
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

In [11]:


DB_PATH = "../db/footbase_big5.db"

# Connect again (read-only mode is fine for testing)
conn = sqlite3.connect(DB_PATH)

In [12]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

      name
0  matches


In [13]:
schema = pd.read_sql_query("PRAGMA table_info(matches);", conn)
print(schema)

    cid                  name     type  notnull dflt_value  pk
0     0                  date     TEXT        0       None   0
1     1             home_team     TEXT        0       None   0
2     2             away_team     TEXT        0       None   0
3     3            home_goals  INTEGER        0       None   0
4     4            away_goals  INTEGER        0       None   0
5     5                result     TEXT        0       None   0
6     6            home_shots  INTEGER        0       None   0
7     7            away_shots  INTEGER        0       None   0
8     8  home_shots_on_target  INTEGER        0       None   0
9     9  away_shots_on_target  INTEGER        0       None   0
10   10             odds_home     REAL        0       None   0
11   11             odds_draw     REAL        0       None   0
12   12             odds_away     REAL        0       None   0
13   13                league     TEXT        0       None   0
14   14                season     TEXT        0       N

In [14]:
df = pd.read_sql_query("SELECT COUNT(*) AS n_matches FROM matches;", conn)
print(df)

sample = pd.read_sql_query("SELECT * FROM matches LIMIT 5;", conn)
display(sample)

   n_matches
0      14359


,date,home_team,away_team,home_goals,away_goals,result,home_shots,away_shots,home_shots_on_target,away_shots_on_target,odds_home,odds_draw,odds_away,league,season
0,11/08/2017,Arsenal,Leicester,4,3,H,27,6,10,3,1.53,4.5,6.50,Premier League,2017/18
1,12/08/2017,Brighton,Man City,0,2,A,6,14,2,4,11.00,5.5,1.33,Premier League,2017/18
2,12/08/2017,Chelsea,Burnley,2,3,A,19,10,6,5,1.25,6.5,15.00,Premier League,2017/18
3,12/08/2017,Crystal Palace,Huddersfield,0,3,A,14,8,4,6,1.83,3.6,5.00,Premier League,2017/18
4,12/08/2017,Everton,Stoke,1,0,H,9,9,4,1,1.70,3.8,5.75,Premier League,2017/18


In [15]:
query = """
SELECT league, season, home_team AS team,
       SUM(home_goals) AS goals_scored
FROM matches
GROUP BY league, season, home_team
ORDER BY goals_scored DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)

,league,season,team,goals_scored
0,Ligue 1,2017/18,Paris SG,70
1,Bundesliga,2020/21,Bayern Munich,64
2,Ligue 1,2018/19,Paris SG,63
3,Premier League,2017/18,Man City,61
4,Premier League,2022/23,Man City,60
5,Premier League,2021/22,Man City,58
6,Premier League,2018/19,Man City,57
7,Premier League,2019/20,Man City,57
8,Bundesliga,2017/18,Bayern Munich,56
9,Bundesliga,2022/23,Dortmund,55
